In [1]:
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import re

df_tweets=pd.read_csv('tweets2_nogreenpass_vaccinatevi_casapound.csv')
df_retweeted=pd.read_csv('retweetted2_nogreenpass_vaccinatevi_casapound.csv')
retweeted=df_retweeted.merge(df_tweets[['id_retweet','user_id_retweet','text_complete','hashtags','colours']],left_on=['id','author_id'],right_on=['id_retweet','user_id_retweet'],how='left')[['id', 'author_id', 'author','n_rt','text_complete','hashtags', 'colours_y']].reset_index(drop=True)
retweeted=retweeted.drop_duplicates(['id','author_id','author']).reset_index(drop=True)
retweeted.head()


,id,author_id,author,n_rt,text_complete,hashtags,colours_y
0,1476028900312551426,2963478049,singerindubai,1,Ho appena visto #DontLookUp incredibile che lo...,['#NoGreenPass'],#fb1239
1,1475926285365006345,1364298062919794688,Lorenzo62752880,106,Anche il sindaco di Verona positivo nonostante...,['#NoGreenPass'],#fb1239
2,1476117524446130180,1296390431190679553,1nessuno100mil2,1,"In Italia si respira una tensione enorme, i n...",['#NoGreenPass'],#fb1239
3,1475960153648291841,1238800491632607232,gianluca826,60,Bassano del Grappa non si ferma. Resistenza do...,['#NoGreenPass'],#fb1239
4,1476095815525584896,1453282570582568960,IlDisilluso,19,"""Sono stati riportati più effetti avversi per ...",[],#aaaacc


In [2]:
Nodes_df6=pd.read_csv('Nodes_df6.csv')
Nodes_df6.columns
retweetted_joined=retweeted.merge(Nodes_df6[['Id','Label','Colour']],left_on='author_id',right_on='Id',how='left')
lista_correzione=['#nogreenpass','#nogreenpass','#vaccinatevi','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#nogreenpass','#vaccinatevi']

index_correzione=retweetted_joined[(retweetted_joined['Colour'].isna()) & (retweetted_joined['hashtags']=='[]')].index
for iter,index in enumerate(index_correzione):
    retweetted_joined.loc[index,'hashtags']=lista_correzione[iter]
for index, row in retweetted_joined.iterrows():
    if row['Colour'] in ['#fb1239','#99cccc']:
        retweetted_joined.at[index,'hashtags']='#NoGreenPass'
        retweetted_joined.at[index,'colours_y']=0
    if row['Colour'] in ['#007474','#b00b69']:
        retweetted_joined.at[index,'hashtags']='#vaccinatevi'
        retweetted_joined.at[index,'colours_y']=1
    if row['Colour'] in ['#042069']:
        retweetted_joined.at[index,'hashtags']='#casapound'
        retweetted_joined.at[index,'colours_y']=2
retweeted=retweetted_joined




In [3]:
nlp = spacy.load('it_core_news_lg') #, disable=['parser', 'ner'])
#lowercase
retweeted['text_preprocessed']=retweeted['text_complete'].apply(lambda x: " ".join(x.lower() for x in x.split()))
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

#pulizia da simboli
new_text=[]
for index,row in retweeted.iterrows():
    l1=[ t for t in row['text_preprocessed'].split() if t.startswith('#') ]
    l2=[ t for t in row['text_preprocessed'].split() if t.startswith('http') ]
    l3=[ t for t in row['text_preprocessed'].split() if t.startswith('@') ]
    stopword=l1+l2+l3
    querywords=row['text_preprocessed'].split()
    resultwords=[word for word in querywords if word.lower() not in stopword]
    result = ' '.join(resultwords)
    new_text.append(result)

retweeted['text_preprocessed']=new_text
#vengono cancellati tutti i termini che non sono alfanumerici o che non corrispondono a spazi vuoti
retweeted['text_preprocessed'] = retweeted['text_preprocessed'].str.replace('[^\w\s]',' ')

stop = stopwords.words('italian')

#rimozione caratteri speciali, stopwords, hyperlink, punteggiatura
retweeted['text_preprocessed_new']=' '

for i in range(0,len(retweeted.text_preprocessed)):
    l1=[ t for t in retweeted['text_preprocessed'][i].split() if t.startswith('http') ]
    stopword=l1+stop
    querywords=retweeted['text_preprocessed'][i].split()
    resultwords=[word for word in querywords if word.lower() not in stopword]
    result = ' '.join(resultwords)
    result=result.replace('[^\w\s]','')
    result=result.translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
    retweeted['text_preprocessed_new'][i]=result

#applicazione per lemmatization con spacy
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

retweeted['text_preprocessed_new']= retweeted['text_preprocessed_new'].apply(space)



C:\Users\Eric\AppData\Local\Temp/ipykernel_15920/3555625496.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  retweeted['text_preprocessed'] = retweeted['text_preprocessed'].str.replace('[^\w\s]',' ')
C:\Users\Eric\AppData\Local\Temp/ipykernel_15920/3555625496.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retweeted['text_preprocessed_new'][i]=result


In [ ]:
#preparazione del lexicon
sentix=pd.read_csv("sentix.csv", header=None,delimiter=';')

#riscaliamo i valori della sentiment come i precedenti valori del lexicon vader
#consideriamo valori estremi di vader lexicon -4 e +4
#consideriamo valori estremi di sentix lexicon -1 e +1

with open("sentix.txt",'w',encoding="utf-8") as txt_file:
    for index,line in sentix.iterrows():
        scaled_score=(((float(line[5]) + 1)/2)*(8))-4
        txt_file.write(str(line[0])+'\t'+str(scaled_score)+'\n')

#creo un dizionario per updatare il lexicon nltk di vader
d= dict([(parola,score) for parola, score in zip(sentix[0],sentix[5])])
sentix_dict=d


#arricchimento vader lexicon per multi-vader
with open('lexicon_vader_enriched.txt','w',encoding="utf-8") as newfile:
    with open('vader_lexicon.txt') as vader_file:
        for line in vader_file:
            newfile.write(line)
    with open('sentix.txt',encoding="utf-8") as sentix_file:
        for line in sentix_file:
            newfile.write(line)
#Algoritmo nltk

In [4]:
from nltk.sentiment import vader

Analyzer = vader.SentimentIntensityAnalyzer()

Analyzer.lexicon.update(sentix_dict)
#vader-multi
from vaderSentiment import vaderSentiment

#non facciamo pulizia del testo sulle emojii
analyzer = vaderSentiment.SentimentIntensityAnalyzer(lexicon_file='lexicon_vader_enriched.txt',emoji_lexicon='emoji_utf8_lexicon.txt')

#il termine "positivo" è stato modificato associandolo a un sentiment negativo
#il termine "negativo" è associato a un sentiment neutro

#vader
from vader_sentiment.vader_sentiment import SentimentIntensityAnalyzer
analyzer2=SentimentIntensityAnalyzer(lexicon_file='sentix.txt',emoji_lexicon='emoji_utf8_lexicon.txt')
#varie prove di confronto tra i 3 algoritmi

NameError: name 'sentix_dict' is not defined

In [5]:
from vaderSentiment import vaderSentiment

#non facciamo pulizia del testo sulle emojii
analyzer = vaderSentiment.SentimentIntensityAnalyzer(lexicon_file='lexicon_vader_enriched.txt',emoji_lexicon='emoji_utf8_lexicon.txt')

In [ ]:
testo=retweeted['text_preprocessed'][16]
#testo='positivo'

print(testo)
print('\n')
print('NLTK: ',Analyzer.polarity_scores(testo))
print('VADER-MULTI: ', analyzer.polarity_scores(testo)) #algoritmo migliore
print('VADER: ', analyzer2.polarity_scores(testo))


In [101]:
index_list=list(retweeted[retweeted.sentiment==0.0].index)

In [102]:
len(index_list)

94

In [100]:
#retweeted=retweeted.reset_index(drop=True)
#retweeted['sentiment']=0.0

for index in index_list:
    if index > 337 :
        try:
            retweeted.at[index,'sentiment']=analyzer.polarity_scores(retweeted.loc[index,'text_preprocessed'])['compound']
        except Exception as e:
            print('exception handled, iteration reached: ',index)
            print('exception: ',e)
            break

exception handled, iteration reached:  428
exception:  HTTP Error 429: Too Many Requests


In [103]:
retweeted.to_csv('retweeted_sentiment.csv')

In [7]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


In [ ]:
no_vax=retweeted[(retweeted.colours_y==0) | (retweeted.colours_y==2)]['text_preprocessed_new']
pro_vax=retweeted[(retweeted.colours_y==1)]['text_preprocessed_new']

#creazione lista novax provax
listone_novax=' '.join(no_vax)
listone_provax=' '.join(pro_vax)

listone_novax= deEmojify(listone_novax)
listone_provax= deEmojify(listone_provax)

#creazione wordcloud con calibrazione dei parametri
wordcloud_novax = WordCloud(width = 2000, height = 1000, random_state=1, prefer_horizontal=1,
                              background_color='white', colormap='Set2', 
                              collocations=False,max_words=75,relative_scaling=0.5,
                              stopwords=['potere','essere','dovere','ogni','tre','poi','cè','me','qui','lamore','volere','fare','cosa','stare','l','e','mai','così','tutto','ciò','avere','molto','anno','tanto','quindi','grazia','dire','solo','andare','giorno','te','prendere','vaccinare','nn','gg','dm','dose','quando','già']).generate(listone_novax)

wordcloud_provax = WordCloud(width = 2000, height = 1000, random_state=1, prefer_horizontal=1,
                               background_color='white', colormap='Set2',
                               collocations=False,max_words=75,relative_scaling=.5,
                               stopwords=['potere','essere','dovere','ogni','tre','poi','cè','me','qui','lamore','volere','fare','cosa','stare','l','e','mai','così','tutto','ciò','avere','molto','anno','tanto','quindi','grazia','dire','solo','andare','giorno','te','prendere','vaccinare','nn','gg','dm','dose','quando','già']).generate(listone_provax)
import matplotlib.pyplot as plt
%matplotlib inline

def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(20, 10))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off")